In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
data=spark.read.csv('C:/Users/dicks/Documents/Food_Inspections.csv', header=True, inferSchema=True)

In [3]:
data.show(5)

+-------------+--------------------+--------------------+---------+-------------+-------------+--------------------+-------+-----+-----+---------------+---------------+---------------+--------------------+-----------+------------+--------------------+
|Inspection ID|            DBA Name|            AKA Name|License #|Facility Type|         Risk|             Address|   City|State|  Zip|Inspection Date|Inspection Type|        Results|          Violations|   Latitude|   Longitude|            Location|
+-------------+--------------------+--------------------+---------+-------------+-------------+--------------------+-------+-----+-----+---------------+---------------+---------------+--------------------+-----------+------------+--------------------+
|      2345239|SAORY RESTAURANT,...|SAORY RESTAURANT,...|  2523066|   Restaurant|Risk 1 (High)|2700 S TRUMBULL AVE |CHICAGO|   IL|60623|      11/7/2019|        Canvass|Out of Business|                null|41.84251235|-87.71122481|(-87.711224813

In [5]:
data.printSchema()

root
 |-- Inspection ID: integer (nullable = true)
 |-- DBA Name: string (nullable = true)
 |-- AKA Name: string (nullable = true)
 |-- License #: integer (nullable = true)
 |-- Facility Type: string (nullable = true)
 |-- Risk: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip: integer (nullable = true)
 |-- Inspection Date: string (nullable = true)
 |-- Inspection Type: string (nullable = true)
 |-- Results: string (nullable = true)
 |-- Violations: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [4]:
#Converting Latitude and Longitude to string type
#Converting inspection date into datetime object
from pyspark.sql.types import StringType
data= data.withColumn("Longitude", data["Longitude"].cast(StringType()))
data= data.withColumn("Latitude", data["Latitude"].cast(StringType()))

In [6]:
#Changing Date column to datetime format and printing the dataframe schema
import pyspark.sql.functions as f
data=(data.withColumn('Inspection Date', f.to_date('Inspection Date','MM/d/yyyy')
                     )
     )
data.printSchema()

root
 |-- Inspection ID: integer (nullable = true)
 |-- DBA Name: string (nullable = true)
 |-- AKA Name: string (nullable = true)
 |-- License #: integer (nullable = true)
 |-- Facility Type: string (nullable = true)
 |-- Risk: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip: integer (nullable = true)
 |-- Inspection Date: date (nullable = true)
 |-- Inspection Type: string (nullable = true)
 |-- Results: string (nullable = true)
 |-- Violations: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



#### Exploratory Analysis: SQL Transformations

In [7]:
data.createOrReplaceTempView('data_view')


In [8]:
#Count of number of inspected facilities per result category
(spark.sql('''
SELECT COUNT(Results) AS Count, Results FROM data_view GROUP BY Results ORDER BY Count DESC
''').show()
)

+------+--------------------+
| Count|             Results|
+------+--------------------+
|105796|                Pass|
| 37868|                Fail|
| 27106|  Pass w/ Conditions|
| 16847|     Out of Business|
|  6234|            No Entry|
|  1877|           Not Ready|
|    68|Business Not Located|
+------+--------------------+



In [9]:
#Count of number of restaurants per result category
(spark.sql('''
SELECT COUNT(*) AS Number_of_Facilities, Risk FROM data_view GROUP BY Risk ORDER BY Number_of_Facilities DESC
''').show()
)

+--------------------+---------------+
|Number_of_Facilities|           Risk|
+--------------------+---------------+
|              140384|  Risk 1 (High)|
|               38227|Risk 2 (Medium)|
|               17082|   Risk 3 (Low)|
|                  73|           null|
|                  30|            All|
+--------------------+---------------+



In [24]:
# Top 10 Violation offences in Chicago
(spark.sql('''
SELECT COUNT(*) AS Number_of_Facilities, Violations FROM data_view WHERE Violations NOT IN ("null") GROUP BY Violations ORDER BY Number_of_Facilities DESC LIMIT 15
''').show()
)

+--------------------+--------------------+
|Number_of_Facilities|          Violations|
+--------------------+--------------------+
|                  11|32. FOOD AND NON-...|
|                  10|45. FOOD HANDLER ...|
|                  10|30. FOOD IN ORIGI...|
|                   7|45. FOOD HANDLER ...|
|                   7|2. FACILITIES TO ...|
|                   7|3. MANAGEMENT, FO...|
|                   7|32. FOOD AND NON-...|
|                   6|3. MANAGEMENT, FO...|
|                   6|40. REFRIGERATION...|
|                   6|38. VENTILATION: ...|
|                   5|2. FACILITIES TO ...|
|                   5|41. PREMISES MAIN...|
|                   5|2. FACILITIES TO ...|
|                   5|45. FOOD HANDLER ...|
|                   5|35. WALLS, CEILIN...|
+--------------------+--------------------+



In [11]:
(spark.sql('''
SELECT Risk, Results, COUNT(*) AS Number_of_Facilities FROM data_view
WHERE Results IN ('Pass', 'Pass w/ Conditions', 'Fail') AND Risk NOT IN ('null','All')
GROUP BY Risk, Results ORDER BY Risk, Number_of_Facilities DESC 
''').show()
)

+---------------+------------------+--------------------+
|           Risk|           Results|Number_of_Facilities|
+---------------+------------------+--------------------+
|  Risk 1 (High)|              Pass|               77869|
|  Risk 1 (High)|              Fail|               26753|
|  Risk 1 (High)|Pass w/ Conditions|               21112|
|Risk 2 (Medium)|              Pass|               20536|
|Risk 2 (Medium)|              Fail|                7352|
|Risk 2 (Medium)|Pass w/ Conditions|                5088|
|   Risk 3 (Low)|              Pass|                7379|
|   Risk 3 (Low)|              Fail|                3736|
|   Risk 3 (Low)|Pass w/ Conditions|                 906|
+---------------+------------------+--------------------+



In [12]:
#Count of Facilities Inspected excluding the null values
(spark.sql('''
SELECT `Facility Type`, COUNT(*) AS Number_of_Facilities FROM data_view
WHERE `Facility Type` NOT IN ("null")
GROUP BY `Facility Type` ORDER BY Number_of_Facilities DESC 
''').show()
)

+--------------------+--------------------+
|       Facility Type|Number_of_Facilities|
+--------------------+--------------------+
|          Restaurant|              130520|
|       Grocery Store|               24936|
|              School|               12110|
|Children's Servic...|                3077|
|              Bakery|                2868|
|Daycare (2 - 6 Ye...|                2690|
|Daycare Above and...|                2369|
|      Long Term Care|                1349|
|            Catering|                1192|
|Mobile Food Dispe...|                 869|
|              Liquor|                 856|
|  Daycare Combo 1586|                 751|
|Mobile Food Preparer|                 631|
|        Golden Diner|                 568|
|            Hospital|                 548|
|           Wholesale|                 535|
|              TAVERN|                 282|
|Daycare (Under 2 ...|                 250|
|       Special Event|                 218|
|Shared Kitchen Us...|          

### Feature Engineering & Model Fitting

In [13]:
#Filtering data: filtering out those food joints whose results were neither pass, pass with conditions or fail
dat=data.filter('Results IN ("Fail","Pass","Pass w/ Conditions")').select('DBA Name','Risk','Violations','Results')
dat.show(5)
 

+--------------------+---------------+--------------------+-------+
|            DBA Name|           Risk|          Violations|Results|
+--------------------+---------------+--------------------+-------+
|    L & M FINE FOODS|  Risk 1 (High)|51. PLUMBING INST...|   Pass|
|CHICAGO'S FINEST ...|  Risk 1 (High)|33. PROPER COOLIN...|   Fail|
|CHICAGO'S FINEST ...|   Risk 3 (Low)|                null|   Fail|
|       NOMAD-CHICAGO|Risk 2 (Medium)|                null|   Pass|
|           STARBUCKS|Risk 2 (Medium)|                null|   Pass|
+--------------------+---------------+--------------------+-------+
only showing top 5 rows



In [20]:
dat.na.drop(subset=["Violations"])

DataFrame[DBA Name: string, Risk: string, Violations: string, Results: string]

In [21]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *
"""Converting the existing dataframe(df) into a new dataframe where each inspection is represented as a label-violations pair. 
In this case, a label of 0.0 represents a failure, a label of 1.0 represents a success, 
and a label of -1.0 represents some results besides those two"""
def labelForResults(s):
    if s == 'Fail':
        return 0.0
    elif s == 'Pass w/ Conditions' or s == 'Pass':
        return 1.0
    else:
        return -1.0
label = UserDefinedFunction(labelForResults, DoubleType())
labeledData = data.select(label(data.Results).alias('label'), data.Violations).where('label >= 0')

In [22]:
#Let's view one row of the labeled data
labeledData.take(1)

[Row(label=1.0, Violations='51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICES - Comments: 3-COMPARTMENT SINK STOPPERS NOT WORKING ALSO OBSERVED 1ST FLOOR PREP AREA HANDWASH BOWL FAUCET LEAKING. MUST PROVIDE WORKING STOPPERS (X3) FOR 3-COMPARTMENT SINK AND MAINTAIN 1ST FLOOR HANDWASH BOWL.')]

In [34]:

from pyspark.ml.feature import Bucketizer, StringIndexer, VectorAssembler, IndexToString
risk_idx = StringIndexer(inputCol = "Risk", outputCol="Risk_idx",handleInvalid="keep")
label_idx = StringIndexer(inputCol="Results", outputCol="Label",handleInvalid="keep")
#Create labels list to decode predictions 
resultLabels= label_idx.fit(dat).labels
riskLabels=risk_idx.fit(dat).labels
riskLabels

['Risk 1 (High)', 'Risk 2 (Medium)', 'Risk 3 (Low)', 'All']

In [35]:
#create a  a single vector combining all input features
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(inputCols=['Label'],outputCol="features") 

In [36]:
train_df,test_df=dat.randomSplit([0.7,0.3])
print(train_df.count()) 
print(test_df.count())


119551
51219


In [37]:
from pyspark.ml.classification import DecisionTreeClassifier, DecisionTreeClassificationModel
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
dt= DecisionTreeClassifier(labelCol="Risk_idx", featuresCol="features")
lc = IndexToString(inputCol="prediction",outputCol="predictedLabel",labels=resultLabels)
dt_pipeline = Pipeline(stages=[label_idx,risk_idx,va,dt,lc])
dtModel =dt_pipeline.fit(train_df) 
resultDF = dtModel.transform(test_df)
#Look for observations where prediction did not match
resultDF.filter("Label != prediction").select( "DBA Name","Label","prediction","Results","predictedLabel").show() 
# Calculation of the accuracy of our classifier
predictionAndLabels = resultDF.select("prediction", "label") 
evaluator = MulticlassClassificationEvaluator().setMetricName("accuracy")
print ("Accuracy:",(evaluator.evaluate(predictionAndLabels))*100,"%")


+--------------------+-----+----------+------------------+--------------+
|            DBA Name|Label|prediction|           Results|predictedLabel|
+--------------------+-----+----------+------------------+--------------+
|    11 DEGREES NORTH|  2.0|       0.0|Pass w/ Conditions|          Pass|
|       11 DINING,LLC|  2.0|       0.0|Pass w/ Conditions|          Pass|
|1492 CUBAN FUSION...|  2.0|       0.0|Pass w/ Conditions|          Pass|
|        1800 LIQUORS|  1.0|       0.0|              Fail|          Pass|
|    2 ASIAN BROTHERS|  1.0|       0.0|              Fail|          Pass|
| 2934 W DIVERSEY LLC|  1.0|       0.0|              Fail|          Pass|
|            3 ABEJAS|  1.0|       0.0|              Fail|          Pass|
|3 JJJ'S BETTER TA...|  1.0|       0.0|              Fail|          Pass|
|4 CAMINOS RESTAUR...|  2.0|       0.0|Pass w/ Conditions|          Pass|
|4 EVER YOUNG DAY ...|  2.0|       0.0|Pass w/ Conditions|          Pass|
|         4 YOLKS,INC|  1.0|       0.0